**Sales Fact**

In [0]:
%sql
INSERT OVERWRITE vsqproject.gold.fact_sales
SELECT
  CAST(date_format(s.sale_date, 'yyyyMMdd') AS INT) AS DateKey,
  s.product_id AS ProductKey,
  s.customer_id AS CustomerKey,
  s.store_id AS StoreKey,
  s.quantity AS Quantity,
  s.sale_amount AS NetSales,
  0.0 AS DiscountAmount,
  (s.sale_amount - (p.price * 0.7 * s.quantity)) AS Profit
FROM vsqproject.silver.silver_sales s
JOIN vsqproject.silver.products_silver p ON s.product_id = p.product_id;

**Product Dimension**

In [0]:
%sql
INSERT OVERWRITE vsqproject.gold.dim_product
SELECT DISTINCT 
  product_id AS ProductKey,
  productname AS ProductName,
  category AS Category,
  brand AS Brand,
  price AS UnitPrice
FROM vsqproject.silver.products_silver;

**Customer Dimension**

In [0]:
%sql
INSERT OVERWRITE vsqproject.gold.dim_customer
SELECT DISTINCT
  customer_id AS CustomerKey,
  CONCAT(firstname, ' ', lastname) AS FullName,
  gender AS Gender,
  CASE 
    WHEN FLOOR(DATEDIFF(current_date(), dob)/365) BETWEEN 18 AND 25 THEN '18-25'
    WHEN FLOOR(DATEDIFF(current_date(), dob)/365) BETWEEN 26 AND 35 THEN '26-35'
    WHEN FLOOR(DATEDIFF(current_date(), dob)/365) BETWEEN 36 AND 50 THEN '36-50'
    ELSE '50+'
  END AS AgeGroup,
  city,
  state
FROM vsqproject.silver.customers_silver;

**Store Dimension**

In [0]:
%sql
INSERT OVERWRITE vsqproject.gold.dim_store
SELECT DISTINCT
  store_id AS StoreKey,
  store_name AS StoreName,
  city,
  state
FROM vsqproject.silver.stores;